In [1]:
# Transformers installation
!pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Su

# Translation

##  Data

In [2]:
import pandas as pd

In [3]:
combined_df_all = pd.read_csv('combined_df.csv', usecols=['EN', 'TI', 'EN_tokenized', 'TI_tokenized'])

In [4]:
combined_df_all.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
combined_df_all["EN"] = combined_df_all["EN"].str.replace(',', '')
combined_df_all["TI"] = combined_df_all["TI"].str.replace(',', '')
combined_df_all.reset_index(drop=True, inplace=True)
combined_df_all.to_csv('dat_1k.csv')

In [5]:
bi_dat_2 = pd.read_csv('bi_dat_2.csv', usecols = ['TI', 'EN', 'TI_tokenized', 'EN_tokenized'])
combined_df = pd.read_csv('combined_df.csv', usecols=['EN', 'TI', 'EN_tokenized', 'TI_tokenized'])

In [6]:
dat_5k = pd.concat([bi_dat_2, combined_df])

In [7]:

dat_5k.reset_index(drop=True, inplace=True)

In [8]:
dat_5k = dat_5k.iloc[:,[1,0,2,3]]

In [9]:
dat_5k.drop(columns=['EN_tokenized', 'TI_tokenized'], inplace=True)
dat_5k["EN"] = dat_5k["EN"].str.replace(',', '')
dat_5k["TI"] = dat_5k["TI"].str.replace(',', '')

In [10]:
dat_5k.reset_index(drop=True, inplace=True)

In [11]:
dat_5k.to_csv('dat_5k.csv')

## Processing

In [13]:
import numpy as np
np.random.seed(1)
text_file = 'dat_1k.csv'
with open(text_file) as f:
    lines = f.read().split('\n')[:-1]

prefix = "translate English to Tigrinya: "
text_pairs = []
for line in lines:
    ind,orig, target = line.split(',')
    orig = orig.replace('"', '')
    target = target.replace('"', '')
    text_pairs.append({'orig': orig, 'target': target})

#Let's create some splits
np.random.shuffle(text_pairs)
num_valid_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_valid_samples
train_pairs = text_pairs[:num_train_samples]
valid_pairs = text_pairs[num_train_samples : num_train_samples + num_valid_samples]
test_pairs = text_pairs[num_train_samples + num_valid_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(valid_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1180 total pairs
826 training pairs
177 validation pairs
177 test pairs


In [14]:
train_pairs[0]

{'orig': 'But you shall go to my country and to my relatives and take a wife for my son Isaac.',
 'target': 'ናብ ሃገረይን ናብ ዓሌተይን ኬድካ ደኣ ንወደይ ይስሃቅ ሰበይቲ ኣምጽኣሉ፡ በሎ።'}

In [15]:
train_list = []

In [16]:
for i in range(len(train_pairs)):
  train_list.append({'id': i, 'translation': {'en': train_pairs[i]['orig'],'ti':train_pairs[i]['target']}})

In [17]:
val_list = []
test_list = []

for i in range(len(valid_pairs)):
  val_list.append({'id': i, 'translation': {'en': valid_pairs[i]['orig'],'ti':valid_pairs[i]['target']}})

for i in range(len(test_pairs)):
  test_list.append({'id': i, 'translation': {'en': test_pairs[i]['orig'],'ti':test_pairs[i]['target']}})

## Preprocess

In [18]:
from transformers import AutoTokenizer


checkpoint = "google/mt5-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Tokenize the input (English) and target (French) separately because you can't tokenize French text with a tokenizer pretrained on an English vocabulary.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [19]:
source_lang = "en"
target_lang = "ti"
prefix = "translate English to Tigrinya: "


def preprocess_function(examples):
    inputs = [prefix + example['translation'][source_lang] for example in examples]
    targets = [example['translation'][target_lang] for example in examples]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs
def preprocess_functionold(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [20]:
import tensorflow as tf
import pandas as pd
from datasets import Dataset

train_data = Dataset.from_list(train_list)
val_data = Dataset.from_list(val_list)
test_data = Dataset.from_list(test_list)

In [21]:
train_data

Dataset({
    features: ['id', 'translation'],
    num_rows: 826
})

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [22]:
tokenized_train = train_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/826 [00:00<?, ? examples/s]

In [23]:
tokenized_val = val_data.map(preprocess_functionold, batched=True)


Map:   0%|          | 0/177 [00:00<?, ? examples/s]

In [25]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [26]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 921.9 kB/s eta 0:00:00


In [27]:
import evaluate

metric = evaluate.load("bleu")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the SacreBLEU score:

In [28]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with Keras, take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-a-tensorflow-model-with-keras)!

</Tip>
To finetune a model in TensorFlow, start by setting up an optimizer function, learning rate schedule, and some training hyperparameters:

In [29]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

Then you can load T5 with [TFAutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.TFAutoModelForSeq2SeqLM):

In [30]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

tf_model.h5:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Convert your datasets to the `tf.data.Dataset` format with [prepare_tf_dataset()](https://huggingface.co/docs/transformers/main/en/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset):

In [31]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)
tf_val_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)


Configure the model for training with [`compile`](https://keras.io/api/models/model_training_apis/#compile-method). Note that Transformers models all have a default task-relevant loss function, so you don't need to specify one unless you want to:

In [32]:
import tensorflow as tf
tf.random.set_seed(1234)
model.compile(optimizer=optimizer)  # No loss argument!

The last two things to setup before you start training is to compute the SacreBLEU metric from the predictions, and provide a way to push your model to the Hub. Both are done by using [Keras callbacks](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/keras_callbacks).

Pass your `compute_metrics` function to [KerasMetricCallback](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks#transformers.KerasMetricCallback):

In [33]:
from transformers.keras_callbacks import KerasMetricCallback
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_val_set)

Specify where to push your model and tokenizer in the [PushToHubCallback](https://huggingface.co/docs/transformers/main/en/main_classes/keras_callbacks#transformers.PushToHubCallback):

Then bundle your callbacks together:

In [35]:
callbacks = [metric_callback]

In [36]:
tokenizer.get_vocab() #['ዊ']

{'lus': 11223,
 '▁pahi': 158057,
 '944.': 106521,
 'jJ': 131703,
 '▁Communications': 61061,
 '蛴': 244818,
 'ttered': 153874,
 '▁поне': 43237,
 'niwers': 75453,
 'colour': 108102,
 '▁લા': 16890,
 '▁behe': 172010,
 'editor': 16972,
 'ецька': 172684,
 'ENA': 41056,
 '▁trat': 78529,
 'नप': 151064,
 'クオ': 200625,
 'uvi': 51544,
 '▁толы': 219201,
 '碱': 235928,
 'ÇÃO': 48326,
 'ecke': 135275,
 '▁pán': 74244,
 '3047': 232980,
 'ител': 63574,
 '▁пед': 165052,
 'նյութ': 109150,
 'atlari': 140850,
 'recipita': 214784,
 '"...': 46789,
 'domnișoare': 179318,
 'КЛА': 233707,
 'ARBE': 172196,
 '权': 20300,
 'ээр': 14351,
 '[25]': 100186,
 '學': 31225,
 'endri': 200536,
 '4087': 236832,
 'ללי': 167874,
 '궈': 241459,
 '▁qa': 6219,
 '\ue4cd': 249608,
 '▁ファッション': 145360,
 '不卡': 35195,
 'пута': 45921,
 '▁Medien': 58428,
 'hulle': 11086,
 '▁między': 47485,
 '▁Strategi': 52531,
 'memora': 94327,
 'ハワイ': 158799,
 'ööriista': 211251,
 'אלק': 237195,
 '▁ADMINISTRA': 149071,
 'certificates': 233066,
 '调控': 218770

Finally, you're ready to start training your model! Call [`fit`](https://keras.io/api/models/model_training_apis/#fit-method) with your training and validation datasets, the number of epochs, and your callbacks to finetune the model:

In [37]:
model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=60) 

Epoch 1/60
206/206 [==============================] - 148s 329ms/step - loss: 20.2292 - val_loss: 11.0997
Epoch 2/60
206/206 [==============================] - 52s 255ms/step - loss: 11.9507 - val_loss: 6.5081
Epoch 3/60
206/206 [==============================] - 52s 255ms/step - loss: 8.4065 - val_loss: 5.9694
Epoch 4/60
206/206 [==============================] - 53s 256ms/step - loss: 7.4266 - val_loss: 5.5904
Epoch 5/60
206/206 [==============================] - 52s 254ms/step - loss: 6.7402 - val_loss: 5.3444
Epoch 6/60
206/206 [==============================] - 53s 255ms/step - loss: 6.3751 - val_loss: 5.3434
Epoch 7/60
206/206 [==============================] - 52s 255ms/step - loss: 6.2286 - val_loss: 5.3169
Epoch 8/60
206/206 [==============================] - 53s 256ms/step - loss: 5.9128 - val_loss: 5.1104
Epoch 9/60
206/206 [==============================] - 52s 254ms/step - loss: 5.6959 - val_loss: 5.0199
Epoch 10/60
206/206 [==============================] - 53s 256ms/step

Once training is completed, your model is automatically uploaded to the Hub so everyone can use it!

<Tip>

For a more in-depth example of how to finetune a model for translation, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation-tf.ipynb).

</Tip>

## Inference

In [38]:
tokenized_test = test_data.map(preprocess_functionold, batched=True)

Map:   0%|          | 0/177 [00:00<?, ? examples/s]

In [39]:
tf_test_set = model.prepare_tf_dataset(
    tokenized_test,
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

In [40]:
train_pairs[0:5]

[{'orig': 'But you shall go to my country and to my relatives and take a wife for my son Isaac.',
  'target': 'ናብ ሃገረይን ናብ ዓሌተይን ኬድካ ደኣ ንወደይ ይስሃቅ ሰበይቲ ኣምጽኣሉ፡ በሎ።'},
 {'orig': 'I will go down now and see whether their deeds are as bad as the reports which have come to me. If not I will know.',
  'target': 'ስለዚ ኸምቲ ናባይ ዝመጸ ጭራሕምራሕ ፈጺሞም ገይሮምዎ ወይ ዘይገበርዎ እንተ ዀይኖም ክፈልጥ፡ ወሪደ ኽርኢ እየ፡ በለ።'},
 {'orig': "Baal Hanan the son of Achbor died and Hadar reigned in his place. The name of his city was Pau. His wife''s name was Mehetabel the daughter of Matred the daughter of Mezahab.",
  'target': 'በዓል-ሓናን ወዲ ኣኽቦር ድማ ሞተ። ሃዳር ከኣ ኣብ ክንዳኡ ነገሰ። ስም ከተማኡ ድማ ጳዑ ነበረ፡ ስም ሰበይቱ ኸኣ ሜሄጣብኤል እዩ፡ ንሳ ጓል ማጥሬድ ጓል ሜ-ዛሃብ እያ።'},
 {'orig': "From now on when you till the ground it won''t yield its strength to you. You shall be a fugitive and a wanderer in the earth.",
  'target': 'ንምድሪ ምስ እትሓርሳ መሊሳ ሓይላ ኣይክትህበካን እያ፡ ኣብ ምድሪ ኰብላልን ቀባሕባሓይን ኩን፡ በለ።'},
 {'orig': 'These are the kings who reigned in the land of Edom before any king reig

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to translate to another language. For T5, you need to prefix your input depending on the task you're working on. For translation from English to French, you should prefix your input as shown below:

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for translation with your model, and pass your text to it:

In [42]:
from transformers import pipeline


You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return the `input_ids` as TensorFlow tensors:

In [43]:
from transformers import AutoTokenizer


Use the [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.TFGenerationMixin.generate) method to create the translation. For more details about the different text generation strategies and parameters for controlling generation, check out the [Text Generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API.

In [44]:
from transformers import TFAutoModelForSeq2SeqLM


Decode the generated token ids back into text:

In [45]:
search_toks = {v:k for k, v in tokenizer.get_vocab().items()}


In [52]:
preds = []
trues = []
for text in test_pairs:
  inputs = tokenizer(text["orig"], return_tensors="tf").input_ids
  outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
  preds.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  trues.append(text["target"])

In [53]:
preds[0:10]

['ኣነ ኣምላኽ ድማ ያእቆብ ሓዉ ሴፍ ኸኣ ኣብ ጽቡቕ ካብ ኢሉ እዚኣብሄር እሞ፡',
 'ንሳራ ድማ ኣብርሃም ኪሕረኻ ድማ፡ ብሓትኡ እሞ፡ ንስኻ ስምዖም ከኣ፡ ምስኡ ሞ',
 'ድማ ዶ',
 'እግዚኣብሄር ሰበይቲ ነበረት እታ ሞተ።',
 'ንጉዱ ድማ ኣምላኽ ምስ ወለደይ ወዲ ማይ ሚእትን እየ፡ ኣብ ክንዲ እዩ ኸኣ፡ በሎ።',
 'ሚእትን ካብኡ ድማ ገጽዋ ትሕቲኡ ዝወለደት ኸኣ፡ ሽዑ ኸኣ፡ ሎሚ ',
 'እቲ ኸኣ ሞተ፡ ነፍሲ ዘላ፡ እሞ፡ እሞ፡ ገበረ፡ ንሱ ኸኣ ካብ ኣውጽኣትን ',
 'ኣብርሃም ኣብ ወዲ ስሙ፡ ከኣ ድማ ኣብኡ ኣብ ወለደ። ኣብ ምድሪ ሰብኣይ ኣዝዮም ኸ',
 'ናይ',
 'ንሱ ድማ ምድሪ ገይሩ፡ ናብቶም ብለይቲ ንራሄል ንሱ ግብጺ ዚነበረ። ንሳ']

In [54]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=preds, references=trues, max_order = 1)
print(results)

{'bleu': 0.04993476957142902, 'precisions': [0.07867867867867868], 'brevity_penalty': 0.6346671094383918, 'length_ratio': 0.6874483897605285, 'translation_length': 1665, 'reference_length': 2422}


In [49]:
preds = []
trues = []
for text in test_pairs:
  inputs = tokenizer(text["orig"], return_tensors="tf").input_ids
  outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=20, top_p=0.95)
  preds.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
  trues.append(text["target"])

In [51]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=preds, references=trues, max_order = 1)
print(results)

{'bleu': 0.04362914737107984, 'precisions': [0.07712248865845756], 'brevity_penalty': 0.5657123898619849, 'length_ratio': 0.6370767960363336, 'translation_length': 1543, 'reference_length': 2422}
